In [1]:
import json
import pandas as pd
from datetime import datetime
import sqlite3 as sl
import re

## Fetch 10 first rows for exploration

In [2]:
# connect to the database
con = sl.connect('arxiv.db')
# build query with 'table'and 'columns'
query = "SELECT * FROM ARXIV LIMIT 5"
# query columns 'categories' and 'versions'
df = pd.read_sql(query, con, coerce_float=False)
df.head()

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{""version"": ""v1"", ""created"": ""Mon, 2 Apr 2007...",2008-11-26,"[[""Balázs"", ""C."", """"], [""Berger"", ""E. L."", """"]..."
1,704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{""version"": ""v1"", ""created"": ""Sat, 31 Mar 200...",2008-12-13,"[[""Streinu"", ""Ileana"", """"], [""Theran"", ""Louis""..."
2,704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,The evolution of Earth-Moon system is descri...,"[{""version"": ""v1"", ""created"": ""Sun, 1 Apr 2007...",2008-01-13,"[[""Pan"", ""Hongjun"", """"]]"
3,704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,None,math.CO,None,We show that a determinant of Stirling cycle...,"[{""version"": ""v1"", ""created"": ""Sat, 31 Mar 200...",2007-05-23,"[[""Callan"", ""David"", """"]]"
4,704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,math.CA math.FA,None,In this paper we show how to compute the $\L...,"[{""version"": ""v1"", ""created"": ""Mon, 2 Apr 2007...",2013-10-15,"[[""Abu-Shammala"", ""Wael"", """"], [""Torchinsky"", ..."


## Query each complete column and perform basic stats

In [3]:
def query_columns(table='ARXIV', columns=['categories', 'versions']):
    """Query 'columns' from 'table' given as parameters.

    Parameters
    -------------------------------------------------------------------------------------
    table: str
        The name of the table to query from.
    columns: list
        The list of columns to query from the table.
    """
    # connect to the database
    con = sl.connect('arxiv.db')
    # build query with 'table'and 'columns'
    query = "SELECT " + "`" + ", ".join(columns) + "`" + " FROM " + table # + " LIMIT 1000"
    # query columns 'categories' and 'versions'
    df = pd.read_sql(query, con, coerce_float=False)
    return df

def col_stats(df):
    """Compute basic statistics on given df for exploration.
    
    Parameters
    -------------------------------------------------------------------------------------
    df: pandas DataFrame
        The DataFrame of one column to compute stats on.
    """
    # number of entries
    total_count = len(df)
    # extract name of the column
    label = df.keys()[0]
    report = "COLUMN {}: \n".format(label.upper())
    # count empty cells in the column
    nan_count = sum(df[label] == '')
    if nan_count != 0:
        report = report + "  - Number of null entries: {} / {}\n".format(nan_count, total_count)
    # number of unique values
    unique_count = len(df[label].unique())
    report = report + "  - Number of unique values: {} / {}\n".format(unique_count, total_count)
    # top10 value counts
    top10_count = list(zip(df[label].value_counts().index[0:20],
                           df[label].value_counts().values[0:20])
                      )
    report = report + "  - Top 20 number of entries per value:\n".format(unique_count)
    for aut in top10_count:
        report = report + "\t {}\n".format(aut)
    print(report)

In [4]:
df_id = query_columns(columns=['id'])
col_stats(df_id)
del df_id

COLUMN ID: 
  - Number of unique values: 1796907 / 1796911
  - Top 20 number of entries per value:
	 ('math-ph/0207039', 2)
	 ('math-ph/0408005', 2)
	 ('math-ph/0409039', 2)
	 ('math-ph/0212014', 2)
	 (1406.6845, 1)
	 (1405.6126, 1)
	 (1801.02556, 1)
	 (1407.7046, 1)
	 ('gr-qc/0302100', 1)
	 (2002.00731, 1)
	 (1606.06185, 1)
	 (1710.0616, 1)
	 (1812.02988, 1)
	 (2002.05238, 1)
	 (1704.02809, 1)
	 (1508.03904, 1)
	 (1502.01745, 1)
	 (1803.05541, 1)
	 ('hep-ex/0212046', 1)
	 (2004.12112, 1)



In [5]:
df_sub = query_columns(columns=['submitter'])
col_stats(df_sub)
del df_sub

COLUMN SUBMITTER: 
  - Number of unique values: 364666 / 1796911
  - Top 20 number of entries per value:
	 ('EPTCS', 2929)
	 ('The CMS Collaboration', 655)
	 ('Atlas Publications', 623)
	 ('EDA Publishing Association', 391)
	 ('Cms Collaboration', 373)
	 ('Delfim F. M. Torres', 367)
	 ('The ATLAS Collaboration', 338)
	 ('Scientific Information Service CERN', 285)
	 ('Alice Publications', 273)
	 ('Rdv Ijcsis', 268)
	 ('Wei Wang', 245)
	 ('Lorenzo Iorio', 244)
	 ('Loet Leydesdorff', 243)
	 ('Shelah Office', 239)
	 ('Taekyun Kim', 237)
	 ('Craig Roberts', 223)
	 ('Ernest Ma', 221)
	 ('Matt Visser', 218)
	 ('Uwe Aickelin', 218)
	 ('Taras Banakh', 214)



In [6]:
df_aut = query_columns(columns=['authors'])
col_stats(df_aut)
del df_aut

COLUMN AUTHORS: 
  - Number of unique values: 1362398 / 1796911
  - Top 20 number of entries per value:
	 ('CMS Collaboration', 992)
	 ('ATLAS Collaboration', 901)
	 ('ALICE Collaboration', 308)
	 ('The BABAR Collaboration, B. Aubert, et al', 263)
	 ('Saharon Shelah', 218)
	 ('Lorenzo Iorio', 214)
	 ('ZEUS Collaboration', 154)
	 ('Miloslav Znojil', 150)
	 ('Shahar Hod', 149)
	 ('The BABAR Collaboration: B. Aubert, et al', 146)
	 ('D0 Collaboration', 141)
	 ('Ernest Ma (UC Riverside)', 140)
	 ('Zhi-Gang Wang', 126)
	 ('Ashoke Sen', 122)
	 ('G.E. Volovik', 121)
	 ('A.G.Ramm', 120)
	 ('CDF Collaboration', 118)
	 ('J. Kluson', 117)
	 ('L3 Collaboration', 116)
	 ('Ali Mostafazadeh', 116)



In [7]:
df_tit = query_columns(columns=['title'])
col_stats(df_tit)
del df_tit

COLUMN TITLE: 
  - Number of unique values: 1793433 / 1796911
  - Top 20 number of entries per value:
	 ('Discussion of: A statistical analysis of multiple temperature proxies:\n  Are reconstructions of surface temperatures over the last 1000 years\n  reliable?', 12)
	 ('Beyond the Standard Model', 11)
	 ('Physics Beyond the Standard Model', 9)
	 ('Neutrino Physics', 9)
	 ('Rare Kaon Decays', 8)
	 ('Discussion of "Least angle regression" by Efron et al', 8)
	 ('A Combination of Preliminary Electroweak Measurements and Constraints on\n  the Standard Model', 7)
	 ('Discussion of: Brownian distance covariance', 7)
	 ('Recent results from NA61/SHINE', 6)
	 ('Discussion: Latent variable graphical model selection via convex\n  optimization', 6)
	 ('Solar Neutrinos', 6)
	 ('Discussion: "A significance test for the lasso"', 6)
	 ('Rejoinder', 6)
	 ('Ultra High Energy Cosmic Rays', 6)
	 ('CP Violation in Hyperon Decays', 6)
	 ('Discussion of "Breakdown and groups" by P. L. Davies and U. Gather'

In [8]:
df_com = query_columns(columns=['comments'])
col_stats(df_com)
del df_com

COLUMN COMMENTS: 
  - Number of unique values: 890631 / 1796911
  - Top 20 number of entries per value:
	 ('10 pages', 6570)
	 ('12 pages', 5736)
	 ('5 pages, 4 figures', 5505)
	 ('8 pages', 5383)
	 ('11 pages', 5347)
	 ('9 pages', 5298)
	 ('14 pages', 5112)
	 ('13 pages', 4922)
	 ('15 pages', 4827)
	 ('4 pages, 4 figures', 4660)
	 ('6 pages', 4642)
	 ('16 pages', 4592)
	 ('7 pages', 4569)
	 ('5 pages', 4419)
	 ('4 pages, 3 figures', 4155)
	 ('18 pages', 4090)
	 ('17 pages', 4065)
	 ('5 pages, 3 figures', 4028)
	 ('4 pages', 3925)
	 ('19 pages', 3701)



In [9]:
df_jou = query_columns(columns=['journal-ref'])
col_stats(df_jou)
del df_jou

COLUMN JOURNAL-REF: 
  - Number of unique values: 687439 / 1796911
  - Top 20 number of entries per value:
	 ("Dans Design, Automation and Test in Europe - DATE'05, Munich :\n  Allemagne (2005)", 128)
	 ('Dans Symposium on Design, Test, Integration and Packaging of\n  MEMS/MOEMS - DTIP 2008, Nice : France (2008)', 77)
	 ('Prog Theor Exp Phys (2020)', 69)
	 ('Dans Symposium on Design, Test, Integration and Packaging of\n  MEMS/MOEMS - DTIP 2007, Stresa, lago Maggiore : Italie (2007)', 66)
	 ('Prog Theor Exp Phys (2019)', 57)
	 ('Prog Theor Exp Phys (2018)', 57)
	 ('Prog Theor Exp Phys (2017)', 53)
	 ('Dans Symposium on Design, Test, Integration and Packaging of\n  MEMS/MOEMS - DTIP 2006, Stresa, Lago Maggiore : Italie (2006)', 49)
	 ("Dans Design, Automation and Test in Europe | Designers'Forum -\n  DATE'05, Munich : Allemagne (2005)", 41)
	 ('Dans Proceedings of the 25th Annual Symposium on the Theoretical\n  Aspects of Computer Science - STACS 2008, Bordeaux : France (2008)', 39)
	 ('

In [10]:
df_doi = query_columns(columns=['doi'])
col_stats(df_doi)
del df_doi

COLUMN DOI: 
  - Number of unique values: 929947 / 1796911
  - Top 20 number of entries per value:
	 ('10.1145/1122445.1122456', 14)
	 ('10.1007/b97728', 12)
	 ('10.1214/009053604000000067', 10)
	 ('10.1016/j.bbr.2011.03.031', 9)
	 ('10.1214/009053604000001138', 8)
	 ('10.1145/1235', 8)
	 ('10.3103/50884591305030022', 8)
	 ('10.1007/b97189', 7)
	 ('10.1007/b13178', 7)
	 ('10.1214/009053604000001048', 6)
	 ('10.1103/PhysRevLett.99.071302', 5)
	 ('10.1140/epjc/s2004-01780-x', 5)
	 ('10.5121/ijcsit', 5)
	 ('10.1088/0953-8984/20/29/293202', 4)
	 ('10.1007/b98716', 4)
	 ('10.1016/S0550-3213(01)00405-9', 4)
	 ('10.1103/PhysRevB.80.024506', 4)
	 ('10.1007/b98411', 4)
	 ('10.1007/978-94-017-7303-4', 4)
	 ('10.4064/fm977-10-2015', 4)



In [11]:
df_rep = query_columns(columns=['report-no'])
col_stats(df_rep)
del df_rep

COLUMN REPORT-NO: 
  - Number of unique values: 159417 / 1796911
  - Top 20 number of entries per value:
	 ('CPH-SYM-DNRF92', 344)
	 ('Roma01.Math', 167)
	 ('CPH-SYM-00', 139)
	 ('Bulletin migration 11/99', 138)
	 ('ISSN 1947 5500', 131)
	 ('Annals migration 4-2001', 98)
	 ('RIKEN-iTHEMS-Report-20', 70)
	 ('JNMP 4/2002 (Article)', 66)
	 ('Roma01.Math.MP', 62)
	 ('Mittag-Leffler-2011spring', 59)
	 ('RIKEN-iTHEMS-Report-19', 56)
	 ('Roma01.math.AG', 53)
	 ('GFL-001', 42)
	 ('JNMP 4/2002 (Letter)', 41)
	 ('Roma01.Math.AP', 38)
	 ('Mittag-Leffler-2010fall', 36)
	 ('ICM-2002', 35)
	 ('SWAT4LS 2010', 34)
	 ('SOAR-GMJT-01', 32)
	 ('Roma01.Math.RT', 31)



In [12]:
df_cat = query_columns(columns=['categories'])
col_stats(df_cat)
del df_cat

COLUMN CATEGORIES: 
  - Number of unique values: 62055 / 1796911
  - Top 20 number of entries per value:
	 ('astro-ph', 86914)
	 ('hep-ph', 73550)
	 ('quant-ph', 53966)
	 ('hep-th', 53287)
	 ('cond-mat.mtrl-sci', 30107)
	 ('cond-mat.mes-hall', 29751)
	 ('cs.CV', 25586)
	 ('gr-qc', 25555)
	 ('math.AP', 24199)
	 ('astro-ph.SR', 22994)
	 ('cond-mat.str-el', 22375)
	 ('astro-ph.GA', 21451)
	 ('math.CO', 21427)
	 ('math.PR', 19008)
	 ('cs.IT math.IT', 18966)
	 ('astro-ph.CO', 18746)
	 ('nucl-th', 17741)
	 ('cond-mat.stat-mech', 17359)
	 ('astro-ph.HE', 17267)
	 ('math.NT', 16456)



In [13]:
df_lic = query_columns(columns=['license'])
col_stats(df_lic)
del df_lic

COLUMN LICENSE: 
  - Number of unique values: 10 / 1796911
  - Top 20 number of entries per value:
	 ('http://arxiv.org/licenses/nonexclusive-distrib/1.0/', 1286517)
	 ('http://creativecommons.org/licenses/by/4.0/', 24529)
	 ('http://creativecommons.org/licenses/by-nc-sa/4.0/', 9522)
	 ('http://creativecommons.org/licenses/by/3.0/', 7930)
	 ('http://creativecommons.org/licenses/by-nc-sa/3.0/', 5886)
	 ('http://creativecommons.org/publicdomain/zero/1.0/', 3608)
	 ('http://creativecommons.org/licenses/by-sa/4.0/', 3207)
	 ('http://creativecommons.org/licenses/publicdomain/', 2480)
	 ('http://creativecommons.org/licenses/by-nc-nd/4.0/', 256)



In [14]:
df_abs = query_columns(columns=['abstract'])
col_stats(df_abs)
del df_abs

COLUMN ABSTRACT: 
  - Number of unique values: 1795292 / 1796911
  - Top 20 number of entries per value:
	 ('  This paper has been withdrawn.\n', 91)
	 ('  This paper has been withdrawn by the author.\n', 62)
	 ('  This paper has been withdrawn by the authors.\n', 38)
	 ('  This paper has been withdrawn\n', 33)
	 ('  This paper has been withdrawn by the author\n', 23)
	 ('  The paper has been withdrawn\n', 16)
	 ('  See hep-ph/0304045\n', 15)
	 ('  Discussion of "A statistical analysis of multiple temperature proxies: Are\nreconstructions of surface temperatures over the last 1000 years reliable?" by\nB.B. McShane and A.J. Wyner [arXiv:1104.4002]\n', 13)
	 ('  The paper has been withdrawn.\n', 13)
	 ('  This paper was withdrawn by the authors.\n', 10)
	 ('  This paper has been withdrawn by the author due to a crucial sign error in\nequation 1\n', 9)
	 ('  This paper has been withdrawn by the authors\n', 8)
	 ("  Discussion of ``Least angle regression'' by Efron et al. [math.ST/0406456]

In [15]:
df_ver = query_columns(columns=['versions'])
col_stats(df_ver)
del df_ver

COLUMN VERSIONS: 
  - Number of unique values: 1789813 / 1796911
  - Top 20 number of entries per value:
	 ('[{"version": "v1", "created": "Wed, 27 Jun 2012 19:59:59 GMT"}]', 103)
	 ('[{"version": "v1", "created": "Tue, 14 Feb 2012 16:41:17 GMT"}]', 82)
	 ('[{"version": "v1", "created": "Thu, 15 Mar 2012 11:17:56 GMT"}]', 78)
	 ('[{"version": "v1", "created": "Sun, 1 Dec 2002 00:00:00 GMT"}]', 33)
	 ('[{"version": "v1", "created": "Wed, 1 Apr 1992 00:00:00 GMT"}]', 22)
	 ('[{"version": "v1", "created": "Wed, 8 Apr 1992 22:58:34 GMT"}]', 18)
	 ('[{"version": "v1", "created": "Thu, 9 Feb 1995 00:00:00 GMT"}]', 18)
	 ('[{"version": "v1", "created": "Fri, 9 Jul 1993 00:00:00 GMT"}]', 17)
	 ('[{"version": "v1", "created": "Thu, 1 Jul 1999 00:00:00 GMT"}]', 17)
	 ('[{"version": "v1", "created": "Wed, 1 Jan 1992 00:00:00 GMT"}]', 17)
	 ('[{"version": "v1", "created": "Thu, 1 Oct 1992 00:00:00 GMT"}]', 16)
	 ('[{"version": "v1", "created": "Sun, 1 Jan 1995 00:00:00 GMT"}]', 13)
	 ('[{"version"

In [16]:
df_upd = query_columns(columns=['update_date'])
col_stats(df_upd)
del df_upd

COLUMN UPDATE_DATE: 
  - Number of unique values: 4095 / 1796911
  - Top 20 number of entries per value:
	 ('2007-05-23', 130748)
	 ('2008-11-26', 33257)
	 ('2009-11-10', 32199)
	 ('2009-10-31', 31908)
	 ('2009-11-07', 29264)
	 ('2009-11-13', 26007)
	 ('2009-11-11', 23375)
	 ('2009-10-30', 12211)
	 ('2009-10-28', 11105)
	 ('2015-05-13', 10143)
	 ('2008-02-03', 6908)
	 ('2009-10-22', 6231)
	 ('2015-05-20', 5946)
	 ('2015-06-11', 5680)
	 ('2015-05-27', 5573)
	 ('2015-06-03', 5410)
	 ('2015-05-19', 5348)
	 ('2015-06-26', 5334)
	 ('2015-06-17', 5250)
	 ('2015-06-16', 5233)



In [17]:
df_aut_p = query_columns(columns=['authors_parsed'])
col_stats(df_aut_p)
del df_aut_p

COLUMN AUTHORS_PARSED: 
  - Number of unique values: 1300964 / 1796911
  - Top 20 number of entries per value:
	 ('[["CMS Collaboration", "", ""]]', 992)
	 ('[["ATLAS Collaboration", "", ""]]', 902)
	 ('[["The BABAR Collaboration", "", ""], ["Aubert", "B.", ""]]', 412)
	 ('[["ALICE Collaboration", "", ""]]', 309)
	 ('[["Shelah", "Saharon", ""]]', 218)
	 ('[["Iorio", "Lorenzo", ""]]', 214)
	 ('[["ZEUS Collaboration", "", ""]]', 156)
	 ('[["Ramm", "A. G.", ""]]', 151)
	 ('[["Znojil", "Miloslav", ""]]', 150)
	 ('[["Hod", "Shahar", ""]]', 149)
	 ('[["D0 Collaboration", "", ""]]', 143)
	 ('[["Ma", "Ernest", "", "UC Riverside"]]', 140)
	 ('[["D0 Collaboration", "", ""], ["Abazov", "V.", ""]]', 127)
	 ('[["Wang", "Zhi-Gang", ""]]', 126)
	 ('[["Volovik", "G. E.", ""]]', 124)
	 ('[["Sen", "Ashoke", ""]]', 122)
	 ('[["Kluson", "J.", ""]]', 121)
	 ('[["Sidharth", "B. G.", ""]]', 121)
	 ('[["CDF Collaboration", "", ""]]', 118)
	 ('[["Moffat", "J. W.", ""]]', 118)



## Explore column categories (target)

In [18]:
df_cat = query_columns(columns=['categories'])
df_cat.head(10)

,categories
0,hep-ph
1,math.CO cs.CG
2,physics.gen-ph
3,math.CO
4,math.CA math.FA
5,cond-mat.mes-hall
6,gr-qc
7,cond-mat.mtrl-sci
8,astro-ph
9,math.CO


#### Count unique values

In [19]:
len(df_cat.categories.unique())

62055

#### Replace by main category

In [21]:
# Define sub categories correspondance
categories = {'cs': ['cs'], 'econ': ['econ'], 'eess': ['eess'], 'math': ['math'], 
              'q-bio': ['q-bio'], 'q-fin': ['q-fin'], 'stat': ['stat'], 
              'physics': ['astro-ph', 'cond-mat', 'gr-qc', 'hep', 'math-ph', 'nlin', 'nucl', 'quant-ph']
             }

# Define function to repalce sub category by main category
def main_cat(cat):
    for key in categories.keys():
        for subcat in categories[key]:
            if subcat in cat:
                return key

In [22]:
# Create new column to apply main_cat
df_cat['main_cat'] = df['categories'].apply(main_cat)

In [23]:
df_cat.head()

,categories,main_cat
0,hep-ph,physics
1,math.CO cs.CG,cs
2,physics.gen-ph,cs
3,math.CO,math
4,math.CA math.FA,math


#### Count unique values in main cat column

In [68]:
len(df_cat.main_cat.unique())

8

In [69]:
df_cat.main_cat.value_counts()

physics    901715
math       416118
cs         400755
stat        30762
q-bio       20717
eess        16725
q-fin        8030
econ         2089
Name: main_cat, dtype: int64

In [70]:
sum(df_cat.main_cat.value_counts().values)

1796911

## Explore column abstract (feature)

In [71]:
df_abs = query_columns(columns=['abstract'])
df_abs.head(10)

,abstract
0,A fully differential calculation in perturba...
1,"We describe a new algorithm, the $(k,\ell)$-..."
2,The evolution of Earth-Moon system is descri...
3,We show that a determinant of Stirling cycle...
4,In this paper we show how to compute the $\L...
5,We study the two-particle wave function of p...
6,A rather non-standard quantum representation...
7,A general formulation was developed to repre...
8,We discuss the results from the combined IRA...
9,Partial cubes are isometric subgraphs of hyp...


#### Extract abstract keywords

In [73]:
from gensim.summarization import keywords

In [79]:
text = df_abs.iloc[0,0]
text

'  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\nmore detailed tests with CDF and DO data. Predictions are shown for\ndistributions of diphoton pairs produced at the energy of the Large Hadron\nCollider (LHC). Distributions of the diphoton pairs from the decay of a Higgs\nboson are contrasted with those produced from QCD processes at the LHC, showing\nthat enhanced sensitivity to the signal can be obtained with judicious\nselection of events.\n'

In [77]:
values = keywords(text=text, split='\n', scores=True)

In [78]:
values

[('gluon', 0.33875403871021675),
 ('lhc', 0.22600055606887032),
 ('perturbative', 0.2083347827247869),
 ('order', 0.2054714891227234),
 ('photon pairs', 0.19791870705781547),
 ('differential', 0.19159566832296288),
 ('good', 0.1915956683229628),
 ('hadron', 0.1571907805770558),
 ('logarithmic', 0.15179478208006933)]

## Explore json file content

In [ ]:
# path to json file
file = 'arxiv-metadata-oai-snapshot.json'
# load json file as chunks iterator
chunks = pd.read_json(file, lines=True, chunksize = 100000)
# initiate empty dataframe to extract chunk
df_explore = pd.DataFrame()
# loop on each chunk
for df in chunks:
    df_explore = df

In [ ]:
for col_id in range(len(df_explore.keys())):
    print(f'- Column \'{df_explore.keys()[col_id]}\': {df_explore.iloc[1,col_id]} --> {type(df_explore.iloc[1,col_id])} \n')

## Populate arxiv database from json file

In [ ]:
# connect to database
con = sl.connect('arxiv.db')

In [ ]:
# path to json file
file = 'arxiv-metadata-oai-snapshot.json'
# load json file as chunks iterator
chunks = pd.read_json(file, lines=True, chunksize = 100000)
# loop on each chunk
for df in chunks:
    # convert 'versions' lists to strings
    df.versions = df.versions.astype('str')
    # replace single quote by double quote in 'versions' strings
    df.versions = df.versions.map(lambda x: x.replace('\'', '\"'))
    # convert 'authors_parsed' lists to strings
    df.authors_parsed = df.authors_parsed.astype('str')
    # replace single quote by double quote in 'authors_parsed' strings
    df.authors_parsed = df.authors_parsed.map(lambda x: x.replace('\'', '\"'))
    # append df chunk to ARXIV table in arxiv.db
    df.to_sql('ARXIV', con, if_exists='append', index=False)

In [ ]:
# checkk ARXIV table content
df_check = pd.read_sql("SELECT * FROM ARXIV LIMIT 5", con)
df_check